In [1]:
import os
import numpy as np
import pandas as pd
import csv
import json
import tqdm

In [2]:
title_akas_df = pd.read_csv('../data/modified-tsvs/title.akas.tsv', sep= '\t')
title_akas_df.head()

/Users/abhinivesh/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,99,tt0000012,10,The Arrival of a Train at La Ciotat,XWW,en,alternative,\N,0
1,114,tt0000012,24,The Arrival of a Train,XWW,en,imdbDisplay,\N,0
2,117,tt0000012,27,The Arrival of a Train,XEU,en,imdbDisplay,\N,0
3,172,tt0000016,1,Boat Leaving the Port,XWW,en,\N,informal title,0
4,261,tt0000029,4,Baby's Meal,XWW,en,\N,informal literal title,0


In [3]:
titles_dict = {}

In [4]:
def create_titles_doc(titles_dict, title_id):

    json_doc = {'id': title_id}

    none_values = ['name', 'region', 'language', 'is_original_title', 'format',
                   'promotional_title', 'original_title', 'is_adult', 'start_year',
                   'end_year', 'runtime_minutes', ]

    list_values = ['types', 'genres', 'directors', 'writers', 'seasons', 'episodes',
                   'categories', 'people', 'jobs', 'characters']

    zero_values = ['genres_count', 'directors_count', 'writers_count', 'seasons_count',
                   'average_rating', 'total_votes', 'categories_count', 'people_count',
                   'jobs_count', 'characters_count', 'episodes_count']

    for value in none_values:
        json_doc[value] = None

    for value in list_values:
        json_doc[value] = []

    for value in zero_values:
        json_doc[value] = 0
        
    titles_dict[ title_id ] = json_doc

    return

def create_people_doc(names_dict, person_id):

    json_doc = {'id': person_id}

    none_values = ['name', 'birth_year', 'death_year']

    list_values = ['professions', 'movies_acted', 'genres', 'directors', 'formats']

    zero_values = ['years_lived', 'professions_count', 'movies_acted_count', 'genres_count',
                   'directors_count', 'formats_count', 'adult_movies_count', 'non_adult_movies_count']

    for value in none_values:
        json_doc[value] = None

    for value in list_values:
        json_doc[value] = []

    for value in zero_values:
        json_doc[value] = 0
        
    names_dict[person_id] = json_doc

    return

In [5]:
csv_folder_path = os.path.join('../data', 'modified-tsvs')
csv_path = os.path.join(csv_folder_path, 'title.akas.tsv')

with open(csv_path) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter= '\t')
    count = 0

    for table_json in csv_reader:
        
        if (table_json['titleId'] not in titles_dict):
            create_titles_doc( titles_dict, table_json['titleId'] )
        
        json_doc = titles_dict[ table_json['titleId'] ]
        json_doc['name'] = table_json['title']
        json_doc['region'] = table_json['region']
        json_doc['language'] = table_json['language']
        
        if (table_json['types'].split(',') != ['\\N']):
            json_doc['types'] = table_json['types'].split(',')
        else:
            json_doc['types'] = []
        
        if (table_json['isOriginalTitle'] == '1'):
            json_doc['is_original_title'] = 1
        else:
            json_doc['is_original_title'] = 0
        
        titles_dict[ table_json['titleId'] ] = json_doc
        
        count += 1
        if (count % 1000 == 0):
            print(count, ' ...Completed')

1000  ...Completed
2000  ...Completed
3000  ...Completed
4000  ...Completed
5000  ...Completed
6000  ...Completed
7000  ...Completed
8000  ...Completed
9000  ...Completed
10000  ...Completed
11000  ...Completed
12000  ...Completed
13000  ...Completed
14000  ...Completed
15000  ...Completed
16000  ...Completed
17000  ...Completed
18000  ...Completed
19000  ...Completed
20000  ...Completed
21000  ...Completed
22000  ...Completed
23000  ...Completed
24000  ...Completed
25000  ...Completed
26000  ...Completed
27000  ...Completed
28000  ...Completed
29000  ...Completed
30000  ...Completed
31000  ...Completed
32000  ...Completed
33000  ...Completed
34000  ...Completed
35000  ...Completed
36000  ...Completed
37000  ...Completed
38000  ...Completed
39000  ...Completed
40000  ...Completed
41000  ...Completed
42000  ...Completed
43000  ...Completed
44000  ...Completed
45000  ...Completed
46000  ...Completed
47000  ...Completed
48000  ...Completed
49000  ...Completed
50000  ...Completed
51000  ..

In [8]:
csv_path = os.path.join(csv_folder_path, 'title.basics.tsv')

with open(csv_path) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter= '\t')
    count = 0

    for table_json in csv_reader:
        
        if (table_json['tconst'] not in titles_dict):
            create_titles_doc( titles_dict, table_json['titleId'] )
        
        json_doc = titles_dict[ table_json['tconst'] ]
        
        json_doc['format'] = table_json['titleType']
        json_doc['promotional_title'] = table_json['primaryTitle']
        json_doc['original_title'] = table_json['originalTitle']
        
        if (table_json['isAdult'] == '1'):
            json_doc['is_adult'] = 1
        else:
            json_doc['is_adult'] = 0
        
        json_doc['start_year'] = table_json['startYear']
        json_doc['end_year'] = table_json['endYear'] if table_json['endYear'] != '\\N' else json_doc['start_year']
        json_doc['runtime_minutes'] = table_json['runtimeMinutes']
        
        try:
            json_doc['start_year'] = int(json_doc['start_year'])
        except:
            json_doc['start_year'] = 2020
        
        try:
            json_doc['end_year'] = int(json_doc['end_year'])
        except:
            json_doc['end_year'] = json_doc['start_year']
        
        try:
            json_doc['runtime_minutes'] = int(json_doc['runtime_minutes'])
        except:
            json_doc['runtime_minutes'] = 1
        
        json_doc['genres'] = table_json['genres'].split(',')
        json_doc['genres_count'] = len(json_doc['genres'])
        
        titles_dict[ table_json['tconst'] ] = json_doc
        
        count += 1
        if (count % 1000 == 0):
            print(count, ' ...Completed')

1000  ...Completed
2000  ...Completed
3000  ...Completed
4000  ...Completed
5000  ...Completed
6000  ...Completed
7000  ...Completed
8000  ...Completed
9000  ...Completed
10000  ...Completed
11000  ...Completed
12000  ...Completed
13000  ...Completed
14000  ...Completed
15000  ...Completed
16000  ...Completed
17000  ...Completed
18000  ...Completed
19000  ...Completed
20000  ...Completed
21000  ...Completed
22000  ...Completed
23000  ...Completed
24000  ...Completed
25000  ...Completed
26000  ...Completed
27000  ...Completed
28000  ...Completed
29000  ...Completed
30000  ...Completed
31000  ...Completed
32000  ...Completed
33000  ...Completed
34000  ...Completed
35000  ...Completed
36000  ...Completed
37000  ...Completed
38000  ...Completed
39000  ...Completed
40000  ...Completed
41000  ...Completed
42000  ...Completed
43000  ...Completed
44000  ...Completed
45000  ...Completed
46000  ...Completed
47000  ...Completed
48000  ...Completed
49000  ...Completed
50000  ...Completed
51000  ..

In [9]:
csv_path = os.path.join(csv_folder_path, 'title.crew.tsv')

with open(csv_path) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter= '\t')
    count = 0

    for table_json in csv_reader:
        
        if (table_json['tconst'] not in titles_dict):
            create_titles_doc( titles_dict, table_json['tconst'] )
        
        json_doc = titles_dict[ table_json['tconst'] ]
        
        if (table_json['directors'].split(',') != ['\\N']):
            json_doc['directors'] = table_json['directors'].split(',')
        
        if (table_json['writers'].split(',') != ['\\N']):
            json_doc['writers'] = table_json['writers'].split(',')
        
        json_doc['directors_count'] = len(json_doc['directors'])
        json_doc['writers_count'] = len(json_doc['writers'])
        
        titles_dict[ table_json['tconst'] ] = json_doc
        
        count += 1
        if (count % 1000 == 0):
            print(count, ' ...Completed')

1000  ...Completed
2000  ...Completed
3000  ...Completed
4000  ...Completed
5000  ...Completed
6000  ...Completed
7000  ...Completed
8000  ...Completed
9000  ...Completed
10000  ...Completed
11000  ...Completed
12000  ...Completed
13000  ...Completed
14000  ...Completed
15000  ...Completed
16000  ...Completed
17000  ...Completed
18000  ...Completed
19000  ...Completed
20000  ...Completed
21000  ...Completed
22000  ...Completed
23000  ...Completed
24000  ...Completed
25000  ...Completed
26000  ...Completed
27000  ...Completed
28000  ...Completed
29000  ...Completed
30000  ...Completed
31000  ...Completed
32000  ...Completed
33000  ...Completed
34000  ...Completed
35000  ...Completed
36000  ...Completed
37000  ...Completed
38000  ...Completed
39000  ...Completed
40000  ...Completed
41000  ...Completed
42000  ...Completed
43000  ...Completed
44000  ...Completed
45000  ...Completed
46000  ...Completed
47000  ...Completed
48000  ...Completed
49000  ...Completed
50000  ...Completed
51000  ..

In [10]:
csv_path = os.path.join(csv_folder_path, 'title.episode.tsv')

with open(csv_path) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter= '\t')
    count = 0

    for table_json in csv_reader:
        
        if (table_json['parentTconst'] not in titles_dict):
            create_titles_doc( titles_dict, table_json['parentTconst'] )
        
        json_doc = titles_dict[ table_json['parentTconst'] ]
        
        season_number = str(table_json['seasonNumber'])
        if (season_number not in json_doc['seasons']):
            json_doc['seasons'].append(season_number)
        json_doc['seasons_count'] = len(json_doc['seasons'])

        episode_number = str(season_number) + '-' + str(table_json['episodeNumber'])
        if (episode_number not in json_doc['episodes']):
            json_doc['episodes'].append(episode_number)
        json_doc['episodes_count'] = len(json_doc['episodes'])
        
        titles_dict[ table_json['parentTconst'] ] = json_doc
        
        count += 1
        if (count % 1000 == 0):
            print(count, ' ...Completed')

1000  ...Completed
2000  ...Completed
3000  ...Completed
4000  ...Completed
5000  ...Completed
6000  ...Completed
7000  ...Completed
8000  ...Completed
9000  ...Completed
10000  ...Completed
11000  ...Completed
12000  ...Completed
13000  ...Completed
14000  ...Completed
15000  ...Completed
16000  ...Completed
17000  ...Completed
18000  ...Completed
19000  ...Completed
20000  ...Completed
21000  ...Completed
22000  ...Completed
23000  ...Completed
24000  ...Completed
25000  ...Completed
26000  ...Completed
27000  ...Completed
28000  ...Completed
29000  ...Completed
30000  ...Completed
31000  ...Completed
32000  ...Completed
33000  ...Completed
34000  ...Completed
35000  ...Completed
36000  ...Completed
37000  ...Completed
38000  ...Completed
39000  ...Completed
40000  ...Completed
41000  ...Completed
42000  ...Completed
43000  ...Completed
44000  ...Completed
45000  ...Completed
46000  ...Completed
47000  ...Completed
48000  ...Completed
49000  ...Completed
50000  ...Completed
51000  ..

408000  ...Completed
409000  ...Completed
410000  ...Completed
411000  ...Completed
412000  ...Completed
413000  ...Completed
414000  ...Completed
415000  ...Completed
416000  ...Completed
417000  ...Completed
418000  ...Completed
419000  ...Completed
420000  ...Completed
421000  ...Completed
422000  ...Completed
423000  ...Completed
424000  ...Completed
425000  ...Completed
426000  ...Completed
427000  ...Completed
428000  ...Completed
429000  ...Completed
430000  ...Completed
431000  ...Completed
432000  ...Completed
433000  ...Completed
434000  ...Completed
435000  ...Completed
436000  ...Completed
437000  ...Completed
438000  ...Completed
439000  ...Completed
440000  ...Completed
441000  ...Completed
442000  ...Completed
443000  ...Completed
444000  ...Completed
445000  ...Completed
446000  ...Completed
447000  ...Completed
448000  ...Completed
449000  ...Completed
450000  ...Completed
451000  ...Completed
452000  ...Completed
453000  ...Completed
454000  ...Completed
455000  ...Co

799000  ...Completed
800000  ...Completed
801000  ...Completed
802000  ...Completed
803000  ...Completed
804000  ...Completed
805000  ...Completed
806000  ...Completed
807000  ...Completed
808000  ...Completed
809000  ...Completed
810000  ...Completed
811000  ...Completed
812000  ...Completed
813000  ...Completed
814000  ...Completed
815000  ...Completed
816000  ...Completed
817000  ...Completed
818000  ...Completed
819000  ...Completed
820000  ...Completed
821000  ...Completed
822000  ...Completed
823000  ...Completed
824000  ...Completed
825000  ...Completed
826000  ...Completed
827000  ...Completed
828000  ...Completed
829000  ...Completed
830000  ...Completed
831000  ...Completed
832000  ...Completed
833000  ...Completed
834000  ...Completed
835000  ...Completed
836000  ...Completed
837000  ...Completed
838000  ...Completed
839000  ...Completed
840000  ...Completed
841000  ...Completed
842000  ...Completed
843000  ...Completed
844000  ...Completed
845000  ...Completed
846000  ...Co

1190000  ...Completed
1191000  ...Completed
1192000  ...Completed
1193000  ...Completed
1194000  ...Completed
1195000  ...Completed
1196000  ...Completed
1197000  ...Completed
1198000  ...Completed
1199000  ...Completed
1200000  ...Completed
1201000  ...Completed
1202000  ...Completed
1203000  ...Completed
1204000  ...Completed
1205000  ...Completed
1206000  ...Completed
1207000  ...Completed
1208000  ...Completed
1209000  ...Completed
1210000  ...Completed
1211000  ...Completed
1212000  ...Completed
1213000  ...Completed
1214000  ...Completed
1215000  ...Completed
1216000  ...Completed
1217000  ...Completed
1218000  ...Completed
1219000  ...Completed
1220000  ...Completed
1221000  ...Completed
1222000  ...Completed
1223000  ...Completed
1224000  ...Completed
1225000  ...Completed
1226000  ...Completed
1227000  ...Completed
1228000  ...Completed
1229000  ...Completed
1230000  ...Completed
1231000  ...Completed
1232000  ...Completed
1233000  ...Completed
1234000  ...Completed
1235000  .

In [11]:
csv_path = os.path.join(csv_folder_path, 'title.principals.tsv')

with open(csv_path) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter= '\t')
    count = 0

    for table_json in csv_reader:
        
        if (table_json['tconst'] not in titles_dict):
            create_titles_doc( titles_dict, table_json['titleId'] )
        
        json_doc = titles_dict[ table_json['tconst'] ]
        
        category = table_json['category']
        if (category not in json_doc['categories']):
            json_doc['categories'].append(category)
        json_doc['categories_count'] = len(json_doc['categories'])

        person = table_json['nconst']
        if (person not in json_doc['people']):
            json_doc['people'].append(person)
        json_doc['people_count'] = len(json_doc['people'])

        job = table_json['job']
        if (job not in json_doc['jobs']):
            json_doc['jobs'].append(job)
        json_doc['jobs_count'] = len(json_doc['jobs'])

        character = table_json['characters']
        if (character not in json_doc['characters']):
            json_doc['characters'].append(character)
        json_doc['characters_count'] = len(json_doc['characters'])
        
        titles_dict[ table_json['tconst'] ] = json_doc
        
        count += 1
        if (count % 1000 == 0):
            print(count, ' ...Completed')

1000  ...Completed
2000  ...Completed
3000  ...Completed
4000  ...Completed
5000  ...Completed
6000  ...Completed
7000  ...Completed
8000  ...Completed
9000  ...Completed
10000  ...Completed
11000  ...Completed
12000  ...Completed
13000  ...Completed
14000  ...Completed
15000  ...Completed
16000  ...Completed
17000  ...Completed
18000  ...Completed
19000  ...Completed
20000  ...Completed
21000  ...Completed
22000  ...Completed
23000  ...Completed
24000  ...Completed
25000  ...Completed
26000  ...Completed
27000  ...Completed
28000  ...Completed
29000  ...Completed
30000  ...Completed
31000  ...Completed
32000  ...Completed
33000  ...Completed
34000  ...Completed
35000  ...Completed
36000  ...Completed
37000  ...Completed
38000  ...Completed
39000  ...Completed
40000  ...Completed
41000  ...Completed
42000  ...Completed
43000  ...Completed
44000  ...Completed
45000  ...Completed
46000  ...Completed
47000  ...Completed
48000  ...Completed
49000  ...Completed
50000  ...Completed
51000  ..

402000  ...Completed
403000  ...Completed
404000  ...Completed
405000  ...Completed
406000  ...Completed
407000  ...Completed
408000  ...Completed
409000  ...Completed
410000  ...Completed
411000  ...Completed
412000  ...Completed
413000  ...Completed
414000  ...Completed
415000  ...Completed
416000  ...Completed
417000  ...Completed
418000  ...Completed
419000  ...Completed
420000  ...Completed
421000  ...Completed
422000  ...Completed
423000  ...Completed
424000  ...Completed
425000  ...Completed
426000  ...Completed
427000  ...Completed
428000  ...Completed
429000  ...Completed
430000  ...Completed
431000  ...Completed
432000  ...Completed
433000  ...Completed
434000  ...Completed
435000  ...Completed
436000  ...Completed
437000  ...Completed
438000  ...Completed
439000  ...Completed
440000  ...Completed
441000  ...Completed
442000  ...Completed
443000  ...Completed
444000  ...Completed
445000  ...Completed
446000  ...Completed
447000  ...Completed
448000  ...Completed
449000  ...Co

795000  ...Completed
796000  ...Completed
797000  ...Completed
798000  ...Completed
799000  ...Completed
800000  ...Completed
801000  ...Completed
802000  ...Completed
803000  ...Completed
804000  ...Completed
805000  ...Completed
806000  ...Completed
807000  ...Completed
808000  ...Completed
809000  ...Completed
810000  ...Completed
811000  ...Completed
812000  ...Completed
813000  ...Completed
814000  ...Completed
815000  ...Completed
816000  ...Completed
817000  ...Completed
818000  ...Completed
819000  ...Completed
820000  ...Completed
821000  ...Completed
822000  ...Completed
823000  ...Completed
824000  ...Completed
825000  ...Completed
826000  ...Completed
827000  ...Completed
828000  ...Completed
829000  ...Completed
830000  ...Completed
831000  ...Completed
832000  ...Completed
833000  ...Completed
834000  ...Completed
835000  ...Completed
836000  ...Completed
837000  ...Completed
838000  ...Completed
839000  ...Completed
840000  ...Completed
841000  ...Completed
842000  ...Co

1185000  ...Completed
1186000  ...Completed
1187000  ...Completed
1188000  ...Completed
1189000  ...Completed
1190000  ...Completed
1191000  ...Completed
1192000  ...Completed
1193000  ...Completed
1194000  ...Completed
1195000  ...Completed
1196000  ...Completed
1197000  ...Completed
1198000  ...Completed
1199000  ...Completed
1200000  ...Completed
1201000  ...Completed
1202000  ...Completed
1203000  ...Completed
1204000  ...Completed
1205000  ...Completed
1206000  ...Completed
1207000  ...Completed
1208000  ...Completed
1209000  ...Completed
1210000  ...Completed
1211000  ...Completed
1212000  ...Completed
1213000  ...Completed
1214000  ...Completed
1215000  ...Completed
1216000  ...Completed
1217000  ...Completed
1218000  ...Completed
1219000  ...Completed
1220000  ...Completed
1221000  ...Completed
1222000  ...Completed
1223000  ...Completed
1224000  ...Completed
1225000  ...Completed
1226000  ...Completed
1227000  ...Completed
1228000  ...Completed
1229000  ...Completed
1230000  .

In [12]:
csv_path = os.path.join(csv_folder_path, 'title.ratings.tsv')

with open(csv_path) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter= '\t')
    count = 0

    for table_json in csv_reader:
        
        if (table_json['tconst'] not in titles_dict):
            create_titles_doc( titles_dict, table_json['titleId'] )
        
        json_doc = titles_dict[ table_json['tconst'] ]
        
        try:
            json_doc['average_rating'] = float(table_json['averageRating'])
        except:
            json_doc['average_rating'] = float(0)
        
        try:
            json_doc['total_votes'] = int(table_json['numVotes'])
        except:
            json_doc['total_votes'] = 0
        
        titles_dict[ table_json['tconst'] ] = json_doc
        
        count += 1
        if (count % 1000 == 0):
            print(count, ' ...Completed')

1000  ...Completed
2000  ...Completed
3000  ...Completed
4000  ...Completed
5000  ...Completed
6000  ...Completed
7000  ...Completed
8000  ...Completed
9000  ...Completed
10000  ...Completed
11000  ...Completed
12000  ...Completed
13000  ...Completed
14000  ...Completed
15000  ...Completed
16000  ...Completed
17000  ...Completed
18000  ...Completed
19000  ...Completed
20000  ...Completed
21000  ...Completed
22000  ...Completed
23000  ...Completed
24000  ...Completed
25000  ...Completed
26000  ...Completed
27000  ...Completed
28000  ...Completed
29000  ...Completed
30000  ...Completed
31000  ...Completed
32000  ...Completed
33000  ...Completed
34000  ...Completed
35000  ...Completed
36000  ...Completed
37000  ...Completed
38000  ...Completed
39000  ...Completed
40000  ...Completed
41000  ...Completed
42000  ...Completed
43000  ...Completed
44000  ...Completed
45000  ...Completed
46000  ...Completed
47000  ...Completed
48000  ...Completed
49000  ...Completed
50000  ...Completed
51000  ..

In [ ]:
with open('../data/titles_en.json', 'w') as fp:
    json.dump(titles_dict, fp)

In [13]:
names_dict = {}

In [14]:
csv_path = os.path.join(csv_folder_path, 'name.basics.tsv')

with open(csv_path) as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter= '\t')
    count = 0

    for table_json in csv_reader:
        
        if (table_json['nconst'] not in names_dict):
            create_people_doc( names_dict, table_json['nconst'] )
        
        json_doc = names_dict[ table_json['nconst'] ]
        
        json_doc['name'] = table_json['primaryName']
        json_doc['birth_year'] = 2020 if table_json['birthYear'] == '\\N' else int(table_json['birthYear'])
        json_doc['death_year'] = 2020 if table_json['deathYear'] == '\\N' else int(table_json['deathYear'])
        json_doc['years_lived'] = json_doc['death_year'] - json_doc['birth_year']
        json_doc['professions'] = table_json['primaryProfession'].split(',')
        json_doc['professions_count'] = len(json_doc['professions'])

        for title_id in table_json['knownForTitles'].split(','):
            
            if (title_id in titles_dict):
                
                title_json = titles_dict[title_id]
                json_doc['movies_acted'].append(title_id)
                json_doc['movies_acted_count'] += 1
                if (title_json['is_adult'] == 0):
                    json_doc['non_adult_movies_count'] += 1
                else:
                    json_doc['adult_movies_count'] += 1

                genres = title_json['genres']
                for genre in genres:
                    if (genre not in json_doc['genres']):
                        json_doc['genres'].append(genre)
                        json_doc['genres_count'] += 1

                directors = title_json['directors']
                for director in directors:
                    if (director not in json_doc['directors']):
                        json_doc['directors'].append(director)
                        json_doc['directors_count'] += 1

                format = title_json['format']
                if (format not in json_doc['formats']):
                    json_doc['formats'].append(format)
                    json_doc['formats_count'] += 1
        
        names_dict[ table_json['nconst'] ] = json_doc
        
        count += 1
        if (count % 1000 == 0):
            print(count, ' ...Completed')

1000  ...Completed
2000  ...Completed
3000  ...Completed
4000  ...Completed
5000  ...Completed
6000  ...Completed
7000  ...Completed
8000  ...Completed
9000  ...Completed
10000  ...Completed
11000  ...Completed
12000  ...Completed
13000  ...Completed
14000  ...Completed
15000  ...Completed
16000  ...Completed
17000  ...Completed
18000  ...Completed
19000  ...Completed
20000  ...Completed
21000  ...Completed
22000  ...Completed
23000  ...Completed
24000  ...Completed
25000  ...Completed
26000  ...Completed
27000  ...Completed
28000  ...Completed
29000  ...Completed
30000  ...Completed
31000  ...Completed
32000  ...Completed
33000  ...Completed
34000  ...Completed
35000  ...Completed
36000  ...Completed
37000  ...Completed
38000  ...Completed
39000  ...Completed
40000  ...Completed
41000  ...Completed
42000  ...Completed
43000  ...Completed
44000  ...Completed
45000  ...Completed
46000  ...Completed
47000  ...Completed
48000  ...Completed
49000  ...Completed
50000  ...Completed
51000  ..

402000  ...Completed
403000  ...Completed
404000  ...Completed
405000  ...Completed
406000  ...Completed
407000  ...Completed
408000  ...Completed
409000  ...Completed
410000  ...Completed
411000  ...Completed
412000  ...Completed
413000  ...Completed
414000  ...Completed
415000  ...Completed
416000  ...Completed
417000  ...Completed
418000  ...Completed
419000  ...Completed
420000  ...Completed
421000  ...Completed
422000  ...Completed
423000  ...Completed
424000  ...Completed
425000  ...Completed
426000  ...Completed
427000  ...Completed
428000  ...Completed
429000  ...Completed
430000  ...Completed
431000  ...Completed
432000  ...Completed
433000  ...Completed
434000  ...Completed
435000  ...Completed
436000  ...Completed
437000  ...Completed
438000  ...Completed
439000  ...Completed
440000  ...Completed
441000  ...Completed
442000  ...Completed
443000  ...Completed
444000  ...Completed
445000  ...Completed
446000  ...Completed
447000  ...Completed
448000  ...Completed
449000  ...Co

In [16]:
titles_dict

{'tt0000012': {'id': 'tt0000012',
  'name': 'The Arrival of a Train',
  'region': 'XEU',
  'language': 'en',
  'is_original_title': 0,
  'format': 'short',
  'promotional_title': 'The Arrival of a Train',
  'original_title': "L'arrivée d'un train à La Ciotat",
  'is_adult': 0,
  'start_year': 1896,
  'end_year': 1896,
  'runtime_minutes': 1,
  'types': ['imdbDisplay'],
  'genres': ['Action', 'Documentary', 'Short'],
  'directors': ['nm0525908', 'nm0525910'],
  'writers': [],
  'seasons': [],
  'episodes': [],
  'categories': ['self', 'director'],
  'people': ['nm2880396',
   'nm9735580',
   'nm0525900',
   'nm9735581',
   'nm0525908',
   'nm0525910',
   'nm9735579',
   'nm9653419'],
  'jobs': ['\\N'],
  'characters': ['["Self"]', '\\N'],
  'genres_count': 3,
  'directors_count': 2,
  'writers_count': 0,
  'seasons_count': 0,
  'average_rating': 7.4,
  'total_votes': 10401,
  'categories_count': 2,
  'people_count': 8,
  'jobs_count': 1,
  'characters_count': 2,
  'episodes_count': 0},


In [17]:
titles_en_df = pd.DataFrame(titles_dict.values())
titles_en_df.head()

,id,name,region,language,is_original_title,format,promotional_title,original_title,is_adult,start_year,...,directors_count,writers_count,seasons_count,average_rating,total_votes,categories_count,people_count,jobs_count,characters_count,episodes_count
0,tt0000012,The Arrival of a Train,XEU,en,0,short,The Arrival of a Train,L'arrivée d'un train à La Ciotat,0.0,1896.0,...,2,0,0,7.4,10401,2,8,1,2,0
1,tt0000016,Boat Leaving the Port,XWW,en,0,short,Barque sortant du port,Barque sortant du port,0.0,1895.0,...,1,0,0,5.9,1210,2,3,1,2,0
2,tt0000029,Baby's Meal,XWW,en,0,short,Baby's Dinner,Repas de bébé,0.0,1895.0,...,1,0,0,5.9,2879,2,4,1,2,0
3,tt0000034,Arrival of a Train at Vincennes Station,XWW,en,0,short,Arrivée d'un train gare de Vincennes,Arrivée d'un train gare de Vincennes,0.0,1896.0,...,1,0,0,5.3,135,1,1,1,1,0
4,tt0000035,The Sprinkler Sprinkled,XWW,en,0,short,Watering the Flowers,L'arroseur,0.0,1896.0,...,1,0,0,5.6,59,1,1,1,1,0


In [18]:
names_en_df = pd.DataFrame(names_dict.values())
names_en_df.head()

,id,name,birth_year,death_year,professions,movies_acted,genres,directors,formats,years_lived,professions_count,movies_acted_count,genres_count,directors_count,formats_count,adult_movies_count,non_adult_movies_count
0,nm0000001,Fred Astaire,1899,1987,"[soundtrack, actor, miscellaneous]","[tt0072308, tt0053137, tt0050419]","[Action, Drama, Thriller, Romance, Sci-Fi, Com...","[nm0347086, nm0006452, nm0002045]",[movie],88,3,3,7,3,1,0,3
1,nm0000002,Lauren Bacall,1924,2014,"[actress, soundtrack]","[tt0037382, tt0117057, tt0038355, tt0071877]","[Adventure, Comedy, Film-Noir, Drama, Romance,...","[nm0001328, nm0000659, nm0001486]",[movie],90,2,4,7,3,1,0,4
2,nm0000003,Brigitte Bardot,1934,2020,"[actress, soundtrack, music_department]","[tt0054452, tt0059956, tt0057345]","[Drama, Adventure, Comedy, Romance]","[nm0167241, nm0001501, nm0000419]",[movie],86,3,3,4,3,1,0,3
3,nm0000004,John Belushi,1949,1982,"[actor, soundtrack, writer]","[tt0078723, tt0072562, tt0080455, tt0077975]","[Action, Comedy, War, Music, Adventure]","[nm0000229, nm3046142, nm0213100, nm0924603, n...","[movie, tvSeries]",33,3,4,5,110,2,0,4
4,nm0000005,Ingmar Bergman,1918,2007,"[writer, director, actor]","[tt0060827, tt0050976, tt0083922, tt0050986]","[Drama, Thriller, Fantasy, Romance]",[nm0000005],[movie],89,3,4,4,1,1,0,4


In [20]:
f = open('../data/names_en.json', 'w')
json.dump( names_en_df.to_dict('records'), f, indent= 4 )
f.close()

In [26]:
titles_en_df['start_year'] = titles_en_df['start_year'].fillna(2020).astype(int)
titles_en_df['end_year'] = titles_en_df['end_year'].fillna(2020).astype(int)
titles_en_df['runtime_minutes'] = titles_en_df['runtime_minutes'].fillna(1).astype(int)
titles_en_df['is_adult'] = titles_en_df['is_adult'].fillna(0).astype(int)

In [28]:
titles_en_df.dtypes

id                    object
name                  object
region                object
language              object
is_original_title      int64
format                object
promotional_title     object
original_title        object
is_adult               int64
start_year             int64
end_year               int64
runtime_minutes        int64
types                 object
genres                object
directors             object
writers               object
seasons               object
episodes              object
categories            object
people                object
jobs                  object
characters            object
genres_count           int64
directors_count        int64
writers_count          int64
seasons_count          int64
average_rating       float64
total_votes            int64
categories_count       int64
people_count           int64
jobs_count             int64
characters_count       int64
episodes_count         int64
dtype: object

In [31]:
f = open('../data/titles_en.json', 'w')
json.dump( titles_en_df.to_dict('records'), f, indent= 4 )
f.close()

In [32]:
names_en_df.dtypes

id                        object
name                      object
birth_year                 int64
death_year                 int64
professions               object
movies_acted              object
genres                    object
directors                 object
formats                   object
years_lived                int64
professions_count          int64
movies_acted_count         int64
genres_count               int64
directors_count            int64
formats_count              int64
adult_movies_count         int64
non_adult_movies_count     int64
dtype: object

In [41]:
n = len(titles_en_df)
chunk_size = n // 10 + 1

for i in range(10):
    temp_df = titles_en_df.iloc[ i * chunk_size: min(i * chunk_size + chunk_size, n) ]
    file_name = '../data/titles_en_' + str(i + 1) + '.json'
    f = open(file_name, 'w')
    json.dump( temp_df.to_dict('records'), f, indent= 4 )
    f.close()
    print(file_name, ' done...')

../data/titles_en_1.json  done...
../data/titles_en_2.json  done...
../data/titles_en_3.json  done...
../data/titles_en_4.json  done...
../data/titles_en_5.json  done...
../data/titles_en_6.json  done...
../data/titles_en_7.json  done...
../data/titles_en_8.json  done...
../data/titles_en_9.json  done...
../data/titles_en_10.json  done...


In [43]:
n = len(names_en_df)
chunk_size = n // 20 + 1

for i in range(20):
    temp_df = names_en_df.iloc[ i * chunk_size: min(i * chunk_size + chunk_size, n) ]
    file_name = '../data/names_en_' + str(i + 1) + '.json'
    f = open(file_name, 'w')
    json.dump( temp_df.to_dict('records'), f, indent= 4 )
    f.close()
    print(file_name, ' done...')

../data/names_en_1.json  done...
../data/names_en_2.json  done...
../data/names_en_3.json  done...
../data/names_en_4.json  done...
../data/names_en_5.json  done...
../data/names_en_6.json  done...
../data/names_en_7.json  done...
../data/names_en_8.json  done...
../data/names_en_9.json  done...
../data/names_en_10.json  done...
../data/names_en_11.json  done...
../data/names_en_12.json  done...
../data/names_en_13.json  done...
../data/names_en_14.json  done...
../data/names_en_15.json  done...
../data/names_en_16.json  done...
../data/names_en_17.json  done...
../data/names_en_18.json  done...
../data/names_en_19.json  done...
../data/names_en_20.json  done...
